In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
import boto3
import io

/usr/local/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


The dataset comes from *Open Food Facts* and was downloaded from: https://www.kaggle.com/openfoodfacts/world-food-facts/home.
'Open Food Facts is a free, open, collbarative database of food products from around the world, with ingredients, allergens, nutrition facts and all the tidbits of information we can find on product labels. Open Food Facts is a non-profit association of volunteers.' Over 5,000 contributors 'have added 100 000+ products from 150 countries using our Android, iPhone or Windows Phone app or their camera to scan barcodes and upload pictures of products and their labels.'

# DATA UPLOAD

In [2]:
food = pd.read_csv(r'/Users/admin/Documents/food.tsv', delimiter = '\t', encoding='ISO-8859-1')

In [ ]:
#s3 data

s3 = boto3.client('s3') 
obj = s3.get_object(Bucket='innawendell', Key='food.tsv') 
food = pd.read_csv(io.BytesIO(obj['Body'].read()), delimiter = '\t', encoding='ISO-8859-1')

In [3]:
food.shape

(356027, 163)

In [4]:
food_sample = food.sample(frac=0.2, replace=True, random_state=108)

In [5]:
food_sample.columns

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name',
       'generic_name', 'quantity',
       ...
       'fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-estimate_100g',
       'collagen-meat-protein-ratio_100g', 'cocoa_100g', 'chlorophyl_100g',
       'carbon-footprint_100g', 'nutrition-score-fr_100g',
       'nutrition-score-uk_100g', 'glycemic-index_100g',
       'water-hardness_100g'],
      dtype='object', length=163)

In [6]:
pd.set_option('display.max_columns', 500)
food_sample.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
43467,41290381257,http://world-en.openfoodfacts.org/product/0041...,usda-ndb-import,1489070248,2017-03-09T14:37:28Z,1489070249,2017-03-09T14:37:29Z,"Mr. Pig, Soda",NaN,NaN,NaN,NaN,"Piggly Wiggly, Piggly Wiggly Company","piggly-wiggly,piggly-wiggly-company",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Carbonated water, high fructose corn syrup, ca...",NaN,NaN,NaN,NaN,NaN,355 ml (1 CAN),NaN,5.0,[ carbonated-water -> en:carbonated-water ] ...,"en:e150a,en:e212,en:e270,en:e338,en:e340,en:e340i","E150a - Plain caramel,E212 - Potassium benzoat...",0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,176.0,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.55,11.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,0.02540,0.010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46446,41345565120,http://world-en.openfoodfacts.org/product/0041...,usda-ndb-import,

# Selecting Retail Country: US

For our analysis, we will only focus on food that is on the market in the US.

In [7]:
food_sample['countries'].value_counts().head(20)

US                34048
France            21197
en:FR              3112
Suisse             2358
Deutschland        1444
EspaÃ±a             735
United Kingdom      690
United States       503
en:CH               453
en:BE               342
en:GB               339
Australia           304
en:FR,France        277
Ð Ð¾ÑÑÐ¸Ñ        261
en:DE               247
en:ES               210
en:US               176
Belgique            159
en:AU               144
Portugal            139
Name: countries, dtype: int64

In [9]:
food_abbr = food_sample[food_sample['countries'].isin(['US', 'United States', 'en:US'])]

In [10]:
food_abbr.shape

(34727, 163)

## Selecting Ingredients and The Target - Energy Value

In [11]:
food_abbr.shape

(34727, 163)

In [12]:
features_energy = food_abbr.loc[:, ['ingredients_text', 'energy_100g']]

In [13]:
features_energy = features_energy.dropna(how='any')

In [14]:
#verify that all the nan values were excluded
features_energy['ingredients_text'].isnull().sum()

0

In [15]:
features_energy['energy_100g'].isnull().sum()

0

In [16]:
features_energy.head()

,ingredients_text,energy_100g
43467,"Carbonated water, high fructose corn syrup, ca...",176.0
46446,"Grapes, water and cane sugar.",297.0
127830,"Whole rolled oats, milled cane sugar, oat syru...",1598.0
293254,"Soy bean (gmo) ,calcium sulfate, water, salt, ...",544.0
116086,"Organic concentrated apple puree, organic conc...",1343.0


# Parsing The Ingredients Text

In [17]:
#making all word lower case
lower = []
for entry in features_energy['ingredients_text']:
    lower.append(entry.lower())

In [18]:
#removing the fullstops from the strings

no_fullstops = []
for entry in lower:
    entry = entry.replace('.', '')
    no_fullstops.append(entry)

In [19]:
#splitting strings at commas
split = []
for entry in no_fullstops:
    entry = entry.split(',')
    split.append(entry)

In [20]:
cleaned = []
for entry in split:
    for item in entry:
        item = item.split("(")
        cleaned.append(item)

In [21]:
cleaned_new = []
for entry in cleaned:
    for item in entry:
        item= item.strip(')')
        cleaned_new.append(item)

In [22]:
cleaned_newer = []
for entry in cleaned_new:
        if entry.count('[') >0:
            entry = entry.split('[') 
            cleaned_newer.append(entry)
        else:
            cleaned_newer.append(entry)

In [23]:
cleaned_newer_one = []
for entry in cleaned_newer:
    if entry.count('and/or') >0:
        entry = entry.split('and/or')
        cleaned_newer_one.append(entry)
    else:
        cleaned_newer_one.append(entry)

In [24]:
unique_words = {}
for entry in cleaned_newer_one:
    if type(entry) == list:
        for word in entry:
            if word.strip() not in unique_words:
                unique_words[word.strip()] = 1
            else:   
                unique_words[word.strip()] += 1
    else:
        if entry.strip() not in unique_words:
            unique_words[entry.strip()] = 1
        else:
            unique_words[entry.strip()] += 1

In [25]:
frequent = {key:value for (key,value) in unique_words.items() if value >= 100}

In [26]:
frequent_vocab = frequent.keys()

In [27]:
for word in frequent_vocab:
    features_energy[word] = features_energy['ingredients_text'].str.contains(word, case=False, regex=False)

In [28]:
features_energy.head()

,ingredients_text,energy_100g,carbonated water,high fructose corn syrup,caramel color,natural and artificial flavor,phosphoric acid,potassium benzoate,a preservative,caffeine,lactic acid,expeller pressed canola oil,honey,molasses,natural flavor,salt,coconut,calcium sulfate,water,sesame oil,ascorbic acid,vitamin c,for color,pectin,organic sunflower oil,popcorn,sucralose,enriched bleached flour,wheat flour,niacin,iron,thiamin mononitrate,vitamin b1,riboflavin,vitamin b2,folic acid],sugar,vegetable oil,,palm,soybean,citric acid,skim milk,baking soda,natural and artificial flavors,carnauba wax,confectioner's glaze,soy flour,sea salt,filtered water,apple juice concentrate,grape juice concentrate,natural flavors,calcium lactate,vinegar,mustard seed,distilled vinegar,garlic powder,spice,cottonseed,sunflower,canola,corn syrup solids,maltodextrin,nonfat dry milk,sodium caseinate,butter,annatto color,mono and diglycerides,milk,soy,sodium acid pyrophosphate,sodium bisulfite,dipotassium phosphate,artificial color,mixed tocopherols,tomatoes,pure cane sugar,organic honey,apple cider vinegar,cinnamon,onion powder,yeast,corn syrup,cocoa butter,nonfat milk,milk fat,soy lecithin,cream,almonds,lecithin,gellan gum,red bell pepper,modified tapioca starch,monosodium glutamate,sodium phosphate,cranberries,sunflower oil,rice flour,corn starch,sunflower lecithin,corn flour,corn meal,enriched flour,reduced iron,thiamine mononitrate,folic acid,partially hydrogenated vegetable oil,palm kernel,cocoa,organic sugar,fruit pectin,tomato puree,tomato paste,soybean oil,romano cheese,rennet,enzymes,modified corn starch,cane sugar,dextrose,malic acid,spices,chili powder,contains one or more of the following: corn,gelatin,sodium lactate,artificial flavors,colors,caramel,titanium dioxide,purified water,unbleached enriched flour,malted barley flour,wheat gluten,dough conditioners,monoglycerides,datem,monocalcium phosphate,calcium propionate,wheat,ferrous sulfate,blueberries,fructose,skim milk powder,cellulose gum,sodium citrate,red 40,blue 1,hazelnuts,cocoa powder,whey powder,lactose,lemon juice,garlic,parsley,oregano,chili pepper,annatto,color,peanuts,cashews,pecans,canola oil,onion,eggs,whey,leavening,organic wheat flour,organic evaporated cane juice,unbleached enriched wheat flour,flour,whole wheat flour,preservatives,sorbic acid,xanthan gum,cornstarch,sodium erythorbate,flavorings,soybeans,cocoa processed with alkali,carrageenan,vanillin,vitamin a palmitate,vitamin d3,red wine vinegar,cheese culture,potato starch,extra virgin olive oil,lemon juice concentrate,enriched wheat flour,thiamine,rolled oats,sesame seeds,apples,chocolate liquor,soya lecithin,safflower,corn oil,palm oil,cottonseed oil,pasteurized milk,whole grain rolled oats,brown rice syrup,soy protein isolate,sodium bicarbonate,calcium chloride,polysorbate 80,yellow 5,olive oil,ginger,milk chocolate,cocoa mass,whole milk powder,emulsifier,glucose syrup,calcium carbonate,brown sugar,onions,natural flavorings,cayenne pepper,peaches,rice starch,guar gum,egg yolks,carob bean gum,cheese cultures,whey protein concentrate,sodium tripolyphosphate,natural & artificial flavors,acesulfame potassium,shrimp,black pepper,turmeric,corn maltodextrin,pineapple,contains 2% or less of: salt,baking powder,cheddar cheese,cultured pasteurized milk,jalapeno peppers,paprika,vitamin b1],vitamin b2],margarine,gum arabic,sunflower seeds,walnuts,pork,hydrolyzed soy protein,sodium phosphates,vitamin e,coconut oil,strawberries,artificial flavor,potassium sorbate,to maintain freshness,fd&c red #40,enriched bleached wheat flour,bleached wheat flour,an emulsifier,stabilizer,modified food starch,locust bean gum,yeast extract,carrots,semolina,durum flour,potatoes,corn,sour cream,disodium inosinate,disodium guanylate,food starch-modified,to protect flavor,aspartame,calcium disodium edta,organic lemon juice concentrate,tomato,vegetable shortening,tomato juice,egg whites,fumaric acid,apple,cumin,tapioca starch,pasteurized cream,invert suga

In [29]:
features_energy.shape

(34304, 489)

In [30]:
type(features_energy.iloc[0, 2])

numpy.bool_

In [31]:
X = features_energy.iloc[:, 2:]
y = features_energy['energy_100g']

# PCA

In [32]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.90)
X_pca = pca.fit_transform(X)

In [33]:
print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    len(pca.explained_variance_ratio_)
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 212


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=108)

In [34]:
rfr_one = RandomForestRegressor(n_estimators = 20, max_depth = 10, n_jobs = -1)
rmse_rfr_one = np.sqrt(np.mean(-cross_val_score(rfr_one, X_pca, y, scoring = 'neg_mean_squared_error')))

In [35]:
print(rmse_rfr_one)

467.59994962624756


# Ridge

In [38]:
from sklearn.metrics import mean_squared_error

In [39]:
rmse_scores = []
alphas = []

for value in [1e-10,   1e-3,  1, 5, 20]:
    ridge = Ridge(alpha=value)
    rmse = np.sqrt(np.mean(-cross_val_score(ridge, X_pca, y, scoring = 'neg_mean_squared_error')))
    rmse_scores.append(rmse)
    alphas.append(value)

In [40]:
df = pd.DataFrame(rmse_scores, columns = ['rmse'])
df['alphas'] = alphas

In [41]:
#alpha 20 is the winner

df.sort_values(by=['rmse'])

,rmse,alphas
4,574.321211,2.000000e+01
3,574.381352,5.000000e+00
2,574.412753,1.000000e+00
1,574.421712,1.000000e-03
0,574.421721,1.000000e-10


# Lasso

In [42]:
alphas_l = []
rmses_l = []

for value in [1e-15, 1e-3,  10]:
    lasso = Lasso(alpha=value)
    rmse_l = np.sqrt(np.mean(-cross_val_score(lasso, X_pca, y, scoring = 'neg_mean_squared_error')))
    rmses_l.append(rmse_l)
    alphas_l.append(value)

In [46]:
df_lasso = pd.DataFrame(rmses_l, columns = ['rmse'])
df_lasso['alphas'] = alphas_l
df_lasso.sort_values(by=['rmse'])

,rmse,alphas
1,574.419668,1.000000e-03
0,574.421721,1.000000e-15
2,630.798976,1.000000e+01


# Elastic Net

In [49]:
from sklearn.model_selection import ParameterGrid

param_grid = {'alpha': [1e-4, 1e-2, 1, 6], 
              'l1_ratio': [0.1, 0.5, 0.95]}

grid = ParameterGrid(param_grid)

rmses_net = []
parameters = []

for params in grid:
    net = ElasticNet(**params)
    rmse_net = np.sqrt(np.mean(-cross_val_score(net, X_pca, y, scoring = 'neg_mean_squared_error')))
    rmses_net.append(rmse_net)
    parameters.append(params)

In [50]:
df_net = pd.DataFrame(rmses_net, columns = ['rmse'])
df_net['alphas'] = parameters
df_net.sort_values(by=['rmse'])

,rmse,alphas
5,574.333776,"{'alpha': 0.01, 'l1_ratio': 0.95}"
0,574.403737,"{'alpha': 0.0001, 'l1_ratio': 0.1}"
1,574.411405,"{'alpha': 0.0001, 'l1_ratio': 0.5}"
2,574.420477,"{'alpha': 0.0001, 'l1_ratio': 0.95}"
4,575.234200,"{'alpha': 0.01, 'l1_ratio': 0.5}"
3,577.023723,"{'alpha': 0.01, 'l1_ratio': 0.1}"
8,603.290625,"{'alpha': 1, 'l1_ratio': 0.95}"
11,672.083734,"{'alpha': 6, 'l1_ratio': 0.95}"
7,683.927571,"{'alpha': 1, 'l1_ratio': 0.5}"
6,713.716626,"{'alpha': 1, 'l1_ratio': 0.1}"


# Random Forest

In [55]:
from sklearn.model_selection import GridSearchCV

rfr = RandomForestRegressor(n_jobs=-1)
params = {'n_estimators': [100, 500,  700], 
        'max_depth': [8, 12, 20],
         }
              
grid = GridSearchCV(estimator = rfr, cv = 3, n_jobs=-1, param_grid = params, scoring = 'neg_mean_squared_error').fit(X_pca, y)
print(grid.best_params_)
print(grid.best_score_)

KeyboardInterrupt: 

for params in grid:
    rfr = RandomForestRegressor(**params, n_jobs=-1)
    rmse_rfr = np.sqrt(np.mean(-cross_val_score(rfr, X_pca, y, scoring = 'neg_mean_squared_error')))
    rmses_rfr.append(rmse_rfr)
    parameters_rfr.append(params)

# XGBoost

In [56]:
params = {'booster': ['gbtree'], 
        'max_depth': [2, 4, 6, 8],
        'learning_rate': [0.1, 0.05, 0.01],
        'subsample': [0.25, 0.5, 1]}

xgb = XGBRegressor(n_jobs = -1)              
grid = GridSearchCV(estimator = xgb, cv=3, n_jobs=-1, param_grid = params,scoring = 'neg_mean_squared_error').fit(X_pca, y)

print(grid.best_params_)
print(grid.best_score_)

KeyboardInterrupt: 

# KNN Regression

In [ ]:
rmse_scores = []
alphas = []

params = {'number': np.arange(10, 100, 10), 
         'weight':['uniform', 'distance']}

knn = KNeighborsRegressor(n_jobs = -1)              
grid = GridSearchCV(estimator = knn, cv=3, n_jobs=-1, param_grid = params,scoring = 'neg_mean_squared_error').fit(X_pca, y)

print(grid.best_params_)
print(grid.best_score_)

# Gradient Boosting

In [ ]:
param_grid = {'loss': ['ls', 'lad', 'huber', 'quantile'], 
              'max_depth': [3, 5, 10, 50],
             'learning_rate': [0.5, 0.1, 0.01]
             'n_estimators': [100, 500, 700]}
             'subsample': [0.25, 0.5, 0.75]

gbr = GradientBoostingRegressor(n_jobs = -1)
grid = GridSearchCV(estimator = gbr, cv=3, n_jobs=-1, param_grid = params, scoring = 'neg_mean_squared_error').fit(X_pca, y)

print(grid.best_params_)
print(grid.best_score_)

param_grid = {'loss': ['ls', 'lad', 'huber', 'quantile'], 
              'max_depth': [3, 5, 10, 50],
             'learning_rate': [0.5, 0.1, 0.01]
             'n_estimators': [100, 500, 700]}
             'subsample': [0.25, 0.5, 0.75]
grid = ParameterGrid(param_grid)

rmses_rfr = []
parameters_rfr = []

for params in grid:
    gbr = GradientBoostingRegressor(**params, n_jobs=-1)
    rmses_rfr = np.sqrt(np.mean(-cross_val_score(rfr, X_pca, y, scoring = 'neg_mean_squared_error')))
    rmses_rfr.append(rmse_gbr)
    parameters_rfr.append(params)